# Huấn luyện có giám sát với SFTTrainer

Bài học này se dạy bạn các huấn luyện mô hình `HuggingFaceTB/SmolLM2-135M` bằng `SFTTrainer` trong thư viện `trl`.  Các cell trong notebook này sẽ chạy và huấn luyện mô hình. Bạn có thể chọn độ khó bằng cách thử nghiệm với các bộ dữ liệu khác nhau.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bài tập: Fine-Tuning SmolLM2 với SFTTrainer</h2>
    <p>Chọn một bộ dự liệu từ Hugging Face hub và huấn luyện một mô hình trên bộ dữ liệu đó. </p>
    <p><b>Các bài tập</b></p>
    <p>🐢 Sử dụng bộ dữ liệu `HuggingFaceTB/smoltalk`</p>
    <p>🐕 Thử nghiệm với bộ dữ liệu `bigcode/the-stack-smol` và huấn luyện một mô hình sinh code trên tập con cụ thể `data/python`.</p>
    <p>🦁 Chọn một bộ dữ liệu liên quan đến một lĩnh vực mà bạn quan tâm</p>
</div>

In [3]:
# Cài đặt các thư viện cần thiết
!pip install transformers datasets trl huggingface_hub

# Đăng nhập vào Hugging Face
from huggingface_hub import login

login()

# Để thuận tiện, bạn có thể tạo một biến môi trường chứa `token hub` của bạn dưới dạng HF_TOKEN

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
# Các thư viện cần thiết
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Tải mô hình và tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Thiết lập định dạng chat
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Đặt tên cho mô hình huấn luyện để lưu &/ tải lên
finetune_name = "SmolLM2-FT-everyday"
finetune_tags = ["smol-course", "module_1"]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

# Sinh văn bản với Mô hình gốc

Ở đây chúng ta sẽ thử nghiệm mô hình gốc chưa được huấn luyện trên định dạng chat.

In [5]:
# Kiểm tra mô hình gốc trước khi huấn luyện
prompt = "Write a haiku about programming"

# Định dạng
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Tạo phản hồi từ mô hình
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
user
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a


## Chuẩn bị dữ liệu

Chúng ta sẽ tải một bộ dữ liệu mẫu và định dạng nó cho việc huấn luyện. Bộ dữ liệu cần được cấu trúc với các cặp đầu vào - đầu ra, trong đó mỗi đầu vào là một chỉ thị và đầu ra là phản hồi mong đợi từ mô hình.

**TRL sẽ định dạng các tin nhắn đầu vào dựa trên định dạng chat của mô hình** Chúng cần được biểu diễn dưới dạng danh sách các từ điển với các khóa: `role` và `content`.

**Ví dụ:**
```sh
[
  {"role": "user", "content": "Hello, how are you?"},
  {"role": "assistant", "content": "I'm doing well, thank you! How can I assist you today?",},
]
```

In [10]:
# Tải dữ liệu mẫu
from datasets import load_dataset

# TODO: Tải bộ dữ liệu thông qua việc điều chỉnh các tham số path và name
ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")

In [11]:
# TODO: 🦁 Nếu dataset của bạn không ở định dạng mà TRL có thể chuyển đổi thành định dạng chat, bạn sẽ cần xử lý nó.
# Tham khảo [Định dạng Chat](../chat_templates.md)

def process_dataset(sample):
    # TODO: 🐢 Chuyển đổi dữ liệu sang định dạng ChatML
    # Sử dụng `tokenizer.apply_chat_template` để áp dụng định dạng hội thoại
    sample = sample.get('messages', [])
    sample = tokenizer.apply_chat_template(sample, tokenize=False, add_generation_prompt=True)
    return {'text': sample}


ds = ds.map(process_dataset)

Map:   0%|          | 0/2260 [00:00<?, ? examples/s]

Map:   0%|          | 0/119 [00:00<?, ? examples/s]

## Điều chỉnh SFTTrainer

Điều chỉnh `SFTTrainer` với các tham số khác nhau giúp điều khiển quá trình huấn luyện trở nên hiệu quả hơn. Các thông số bao gồm
- Số bước huấn luyện (steps)
- Kích thước batch (batch size)
- Tốc độ học (learning rate)
- Chiến lược đánh giá mô hình (evaluation strategy)

Ngoài ra, còn rất nhiều thông số khác, bạn có thể tham khảo thêm ở [SFTTrainer](https://huggingface.co/docs/trl/sft_trainer)

Điều chỉnh các tham số này dựa trên yêu cầu cụ thể và tài nguyên tính toán của bạn.

In [12]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=500,  # Frequency of saving model checkpoints
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id=finetune_name,  # Set a unique name for your model
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
    eval_dataset=ds["test"],
)


# TODO: 🦁 🐕 căn chỉnh các tham số SFTTrainer với bộ dữ liệu bạn đã chọn.
# Ví dụ, nếu bạn đang sử dụng bộ `bigcode/the-stack-smol`, bạn sẽ cần chọn cột `content`

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/2260 [00:00<?, ? examples/s]

Map:   0%|          | 0/119 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


## Huấn luyện Mô hình

Với trainer đã được điều chỉnh, chúng ta có thể tiến hành huấn luyện mô hình. Quá trình huấn luyện sẽ bao gồm
- Lặp qua bộ dữ liệu
- Tính toán loss
- Cập nhật các tham số của mô hình để giảm thiểu loss này.

In [13]:
# Huấn luyện mô hình
trainer.train()

# Lưu mô hình
trainer.save_model(f"./{finetune_name}")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss
50,1.065700,1.158981
100,1.111600,1.124064
150,1.062400,1.095485
200,1.048200,1.079699
250,1.041200,1.070457
300,1.029200,1.061473
350,1.003400,1.054751
400,1.006500,1.050794
450,1.021100,1.042637
500,1.076200,1.033726


In [14]:
# Đưa mô hình lên Hugging Face Hub
trainer.push_to_hub(tags=finetune_tags)

model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1734329611.876bc4d93c2a.1087.0:   0%|          | 0.00/32.4k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.56k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/quyettv/SmolLM2-FT-everyday/commit/cb85819ef0da904c950220f29adbe8eb84ba5956', commit_message='End of training', commit_description='', oid='cb85819ef0da904c950220f29adbe8eb84ba5956', pr_url=None, repo_url=RepoUrl('https://huggingface.co/quyettv/SmolLM2-FT-everyday', endpoint='https://huggingface.co', repo_type='model', repo_id='quyettv/SmolLM2-FT-everyday'), pr_revision=None, pr_num=None)

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bài tập thêm: Sinh văn bản với mô hình vừa được huấn luyện</h2>
    <p>🐕 Sử dụng mô hình đã được huấn luyện để sinh ra phản hồi, giống như với ví dụ ban đầu.</p>
</div>

In [15]:
# Kiểm tra mô hình đã được huấn luyện trên cùng một yêu cầu

# Kiểm tra mô hình gốc trước khi huấn luyện
prompt = "Write a haiku about programming"

# Định dạng chat
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Sinh phản hồi từ mô hình
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# TODO: sử dụng mô hình đã được huấn luyện để sinh phản hồi, giống như với ví dụ.
outputs = model.generate(**inputs, max_new_tokens=100)
print("After training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

After training:
user
Write a haiku about programming
assistant
Hello! How can I help you today? I'm going to write a haiku about programming. What's programming? It's a type of art where you create a sequence of steps to solve a problem. Think of it like a recipe. You follow the steps, and then you see the result. Can you think of any programming languages? Python, Java, or JavaScript are all popular ones. What's the most popular programming language right now? Python is very popular,


## 💐 Chúc mừng bạn. Bạn đã hoàn thành!

Bài tập này đã cung cấp hướng dẫn từng bước để bạn huấn luyện được mô hình `HuggingFaceTB/SmolLM2-135M` sử dụng `SFTTrainer`. Bằng cách làm theo các bước này, bạn có thể điều chỉnh mô hình để thực hiện các tác vụ cụ thể hiệu quả hơn. Nếu bạn muốn tiếp tục làm việc với khóa học này, đây là một số bước bạn có thể thử:

- Thử notebook này ở mức độ khó hơn
- Review PR của học viên khác
- Cải thiện tài liệu khóa học thông qua Issue hoặc PR.